## Домашнее задание по теме: "Классификация: Логистическая регрессия и SVM"

Описание задания:

В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

### Загрузите данные. Используйте датасет с ирисами. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

In [ ]:
import copy
import numpy as np
import pandas as pd

In [ ]:
# загружаем датасет с видами ириса
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
# находим индекс элемента с видом ириса setosa
np.where(iris.target_names == 'setosa') 

(array([0]),)

In [ ]:
# создаем датафрейм
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['target'] = iris.target
# избавляемся от ириса setosa
df = df[df.target != 0]
# приводим виды ириса к бинарному виду
df.target = (df.target - 1)

# проверяем случайную выборку
df.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
126,6.2,2.8,4.8,1.8,1
96,5.7,2.9,4.2,1.3,0
113,5.7,2.5,5.0,2.0,1
99,5.7,2.8,4.1,1.3,0
50,7.0,3.2,4.7,1.4,0


In [ ]:
X_raw = df.drop(columns=['target']).to_numpy ()
y = df['target'].to_numpy ()

### 2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции.

*Оформите в виде класса с методами.

In [ ]:
def standardize_intercept(X):
    '''Функция для стандартизации данных'''
    std_X = (X - X.mean(0) ) / X.std(0)
    print(f'Среднее отклонение:\n{std_X.mean(axis=0)}\n')
    print(f'Стандартное отклонение:\n{std_X.std(axis=0)}')
    return std_X

In [ ]:
class LogisticRegression:
    def __init__(self, learning_rate=0.001, epoch=10000, intercept=False, gamma=0.0001, optimization='gd', alpha=0.9, beta_1=0.9, beta_2=0.999, epsilon=1e-8):
        # инициализация скорости обучения, количества итераций, степени точности, алгоритма оптимизации, весов
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.intercept = intercept
        self.optimization = optimization
        self.gamma = gamma
        self.weights = None
        self.alpha = alpha
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon

    def add_intercept(self, X):
        # смещение
        return np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

    def sigmoid(self, x):
        # вычисление сигмоиды
        return 1 / (1 + np.exp(-x))

    def logloss(self, y, y_pred):
        # вычисление средней кросс-энтропийной потери
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def fit(self, X, y):
        # if self.intercept:
        #     X = self.add_intercept(X)

        # заполнение весов нулями
        self.weights = np.zeros(X.shape[1])
        _weights = self.weights

        v = np.zeros(self.weights.shape)
        mw = np.zeros(self.weights.shape)
        vw = np.zeros(self.weights.shape)

        for i in range(self.epoch):
            # итерируем для нахождения минимума
            y_pred = self.sigmoid(X @ self.weights)

            # вычисление градиента
            gradient = X.T @ (y_pred - y) / y.size

            if self.optimization == 'gd':
                # оптимизация по методу градиентного спуска
                self.weights = _weights - self.learning_rate * gradient

            if self.optimization == 'rmsprop':
                # оптимизация по методу скользящего среднего
                v = self.alpha * v + (1 - self.alpha) * gradient**2
                self.weights = _weights - self.learning_rate * gradient / (np.sqrt(v) + self.epsilon)

            if self.optimization == 'nadam':
                # оптимизация ускоренного по Нестерову метода адаптивной оценки моментов
                mw = self.beta_1 * mw + (1 - self.beta_1) * gradient
                vw = self.beta_2 * vw + (1 - self.beta_2) * gradient**2
                _mw = mw / (1 - self.beta_1**(i + 1))
                _vw = vw / (1 - self.beta_2**(i + 1))
                self.weights -= self.learning_rate * _mw / (np.sqrt(_vw) + self.epsilon)

            if np.linalg.norm(_weights - self.weights) <= self.gamma:
                # выход из цикла при достижении необходимой степени точности
                print(f'Количество итераций для нахождения оптимальных весов (при epsilon = {self.gamma}): {i}')
                break

    def predict_proba(self, X):
        # предсказание вероятностей
        return self.sigmoid(X @ self.weights)

    def predict(self, X, threshold=0.5):
        # классификация
        return self.predict_proba(X) >= threshold

    def accuracy(self, X, y):
        # вычисление accuracy
        print(f'Accuracy: {(self.predict(X) == y).mean()}')


In [ ]:
X = standardize(X_raw)

Среднее отклонение:
[ 3.28848060e-15 -3.63598041e-15 -5.42066392e-16 -1.28397293e-15]

Стандартное отклонение:
[1. 1. 1. 1.]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

### Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.

In [ ]:
# обучение модели методом градиентного спуска
clf_gd = LogisticRegression() 
%time clf_gd.fit(X_train, y_train)
clf_gd.accuracy(X_train, y_train)

CPU times: user 143 ms, sys: 316 µs, total: 143 ms
Wall time: 143 ms
Accuracy: 0.9333333333333333


In [ ]:
# проверка на тестовой выборке
clf_gd.accuracy(X_test, y_test)

Accuracy: 0.88


### Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).

In [ ]:
# обучение модели методом RMSProp
clf_rmsp = LogisticRegression(optimization='rmsprop') 
%time clf_rmsp.fit(X_train, y_train)
clf_rmsp.accuracy(X_train, y_train)

CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 195 ms
Accuracy: 0.8


In [ ]:
# проверка на тестовой выборке
clf_rmsp.accuracy(X_test, y_test)

Accuracy: 0.8


### Повторите для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).

In [ ]:
# обучение модели методом Nadam
clf_nadam = LogisticRegression(optimization='nadam')
%time clf_nadam.fit(X_train, y_train)
clf_nadam.accuracy(X_train, y_train)

Количество итераций для нахождения оптимальных весов (при epsilon = 0.0001): 0
CPU times: user 822 µs, sys: 933 µs, total: 1.75 ms
Wall time: 1.76 ms
Accuracy: 0.8


In [ ]:
# проверка на тестовой выборке
clf_nadam.accuracy(X_test, y_test)

Accuracy: 0.8


### Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

|Метод | Метрика | Время работы|
|:-  |:-:| -:|
|GD | 0.9333 |  143ms|
|RMSProp | 0.8  | 195ms|
|Nadam | 0.8 | 1.76ms|

Реализована логистическая регрессия с 3-мя методами оптимизации.